<a href="https://colab.research.google.com/github/palak0626/predictive-analysis/blob/main/topsis_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd
import numpy as np
import sys
import os

In [68]:
%%writefile topsis.py
import pandas as pd
import numpy as np
import sys
import os

def run_topsis():
    if len(sys.argv) != 5:
        print("Error: Wrong number of parameters.")
        print("Usage: python topsis.py <InputDataFile> <Weights> <Impacts> <OutputResultFileName>")
        return

    file_path = sys.argv[1]
    weights_raw = sys.argv[2]
    impacts_raw = sys.argv[3]
    out_path = sys.argv[4]

    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' not found.")
        return

    try:

        df = pd.read_csv(file_path)

        if len(df.columns) < 3:
            print("Error: Input file must contain three or more columns.")
            return

        try:
            data = df.iloc[:, 1:].values.astype(float)
        except ValueError:
            print("Error: From 2nd column onwards, values must be numeric.")
            return

        weights = [float(w) for w in weights_raw.split(',')]
        impacts = impacts_raw.split(',')

        if len(weights) != data.shape[1] or len(impacts) != data.shape[1]:
            print("Error: Number of weights, impacts, and numeric columns must be the same.")
            return

        for imp in impacts:
            if imp not in ['+', '-']:
                print("Error: Impacts must be either '+' or '-'.")
                return


        norm = data / np.sqrt((data**2).sum(axis=0))
        weighted_norm = norm * weights

        ideal_best = []
        ideal_worst = []
        for i, imp in enumerate(impacts):
            if imp == '+':
                ideal_best.append(max(weighted_norm[:, i]))
                ideal_worst.append(min(weighted_norm[:, i]))
            else:
                ideal_best.append(min(weighted_norm[:, i]))
                ideal_worst.append(max(weighted_norm[:, i]))

        dist_best = np.sqrt(((weighted_norm - ideal_best)**2).sum(axis=1))
        dist_worst = np.sqrt(((weighted_norm - ideal_worst)**2).sum(axis=1))
        score = dist_worst / (dist_best + dist_worst)

        df['Topsis Score'] = score
        df['Rank'] = df['Topsis Score'].rank(ascending=False).astype(int)

        df.to_csv(out_path, index=False)
        print(f"Success: Results saved to {out_path}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    run_topsis()

Overwriting topsis.py


In [69]:
import pandas as pd
pd.read_excel('/content/data.xlsx').to_csv('data.csv', index=False)

In [70]:
!python topsis.py data.csv "1,1,1,1,1" "+,+,+,+,-" 102303826-result.csv

Success: Results saved to 102303826-result.csv
